In [1]:
import pandas as pd
import numpy as np
# from random import shuffle
from DiyLSTMmod import *
from sklearn.model_selection import train_test_split
from optimizationAlgorithm import *
import shutil
from sklearn.utils import shuffle
tf.random.set_seed(123)
shutil.rmtree('models')  
os.mkdir('models')
print(tf.__version__)
print(keras.__version__)

Using TensorFlow backend.


2.2.0
2.3.1


In [2]:
def FReIndex(colIndex,reIndex):
    dirindex = {}
    lisindex = []
    for i in range(len(colIndex)):
        lisindex.append(i)
    dirindex = dict(zip(lisindex,colIndex))
    colIndex = []
    for i in range(len(reIndex)):
        for key in dirindex:
            if int(key) + 1 == int(reIndex[i]):
                colIndex.append(dirindex[key])
                break
    #print(colIndex)
    return colIndex

In [3]:
def creat_dataset_X_Y(lis,path):
    df = pd.read_excel(path)
    df = shuffle(df)
    df.reset_index(drop = True,inplace = True)
    #print(df)
    Lisclass = df['Class'].values
    LisClassNp = []
    for i in range(len(Lisclass)):
        #if (Lisclass[i] == 'Kecimen'):  葡萄干
        if (Lisclass[i] == 'Kirmizi_Pistachio'): #开心果
            LisClassNp.append([0,1])
        else:
            LisClassNp.append([1,0])
    Y = np.array(LisClassNp)
    df = df.drop(columns='Class')
    colIndex = ['Area', 'Perimeter', 'Major_Axis', 'Minor_Axis', 'Eccentricity',
       'Eqdiasq', 'Solidity', 'Convex_Area', 'Extent', 'Aspect_Ratio',
       'Roundness', 'Compactness', 'Shapefactor_1', 'Shapefactor_2',
       'Shapefactor_3', 'Shapefactor_4']#开心果 16
    #colIndex = ['Area', 'Perimeter', 'Major_Axis', 'Minor_Axis', 'Eccentricity',
                #'Eqdiasq', 'Solidity']#开心果
    #葡萄干#colIndex = ['Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity','ConvexArea', 'Extent', 'Perimeter']
    colIndex = FReIndex(colIndex,lis)
    df = df[colIndex]
    X = df.values
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    X = scaler.fit_transform(X)
    #拆分训练集和测试集
    x_train, X_test, y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=40)
    X_train = X
    Y_train = Y
    X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
    X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

    X_train = tf.cast(X_train, dtype='float32')
    X_test = tf.cast(X_test, dtype='float32')
    Y_train = tf.cast(Y_train, dtype='float32')
    Y_test = tf.cast(Y_test, dtype='float32')
    return X_train,Y_train,X_test,Y_test
    

In [4]:
def enSimpleCode(lis):
    #print('lis',lis)
    #Pistachio_28_Features_Dataset
    #X_train, Y_train,X_test,Y_test = creat_dataset_X_Y(lis,'./dataset/Raisin_Dataset.xlsx') 葡萄干
    X_train, Y_train,X_test,Y_test = creat_dataset_X_Y(lis,'./dataset/Pistachio_28_Features_Dataset.xlsx')#开心果
    checkpoint_path = "./models_Pistachio/{categorical_accuracy:.5f}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    train_acc = []
    valid_acc = []
    for i in range(1):
        t,v = k_fold(10, X_train, Y_train,100,128)
        train_acc.append(t)
        valid_acc.append(v)
    #shutil.rmtree('./models')
    Note = open('restultOptimization.txt',mode='a+')
    Note.write(f'{lis}' + ',' + f'{valid_acc}\n')
    Note.close()
    # print('训练集：',train_acc)
    # print('验证集:',valid_acc)

    # print('训练集平均，验证集平均：',ave_list(train_acc),ave_list(valid_acc))
    return valid_acc

In [5]:
lis = [x for x in range(1,17)]
print(lis)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


In [6]:
#训练新模型
#enSimpleCode([1,2,3,4,5,6,7]) 葡萄干
#CNN+LSTM 0.8972222328186035  葡萄干
#enSimpleCode([1,2,3,4,5,6,7])  #开心果
#CNN+LSTM [0.8720930218696594] [0.8988372206687927]
#11.20 LSTM  [0.8895348906517029] 
#enSimpleCode(lis)
#11.22 LSTM_16 开心果 [0.8872093081474304]  [0.897927463054657]

In [7]:
def enSimpleCodeLis(lis):
    net2 =  my_model((sequence_lenth,input_size), name='MyModelMLP2')
    net2.compile(loss='categorical_crossentropy',#categorical_crossentropy', binary_crossentropy
        optimizer='adam', metrics=['categorical_accuracy'],run_eagerly=True)  
    net2.load_weights('./0.89729.ckpt')
    X_train, Y_train,X_test,Y_test = creat_dataset_X_Y(lis,'./dataset/Pistachio_28_Features_Dataset.xlsx')
    history = net2.evaluate(X_test,Y_test,verbose=0)
    Note = open('restultOptimization.txt',mode='a+')
    Note.write(f'{lis}' + ',' + f'{history[1]}\n')
    Note.close()
    return -history[1]

In [8]:
Note = open('restultOptimization.txt',mode='w')
Note.close()
ColIndex = ['Area', 'Perimeter', 'Major_Axis', 'Minor_Axis', 'Eccentricity',
       'Eqdiasq', 'Solidity', 'Convex_Area', 'Extent', 'Aspect_Ratio',
       'Roundness', 'Compactness', 'Shapefactor_1', 'Shapefactor_2',
       'Shapefactor_3', 'Shapefactor_4']#开心果
#ColIndex = ['Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity','ConvexArea', 'Extent', 'Perimeter'] 葡萄干
optAlgo = OptimizationAlgorithm(ColIndex)
optAlgo.ga(enSimpleCodeLis, len(ColIndex), 6000, 1)
#11-11 11:09 CNN+LSTM
#11.20 16:37 LSTM

MemoryError: 

In [ ]:
# shutil.rmtree('models')  

In [ ]:
# os.mkdir('models')

80---84----82

In [ ]:
# enSimpleCode([3, 6, 7, 4, 2, 5, 1])

In [ ]:
# shutil.rmtree('models')  
# os.mkdir('models')
# Note = open('restultOptimization.txt',mode='w')
# Note.close()
# ColIndex = ['Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity','ConvexArea', 'Extent', 'Perimeter']
# optAlgo = OptimizationAlgorithm(ColIndex)
# optAlgo.ga(enSimpleCode, len(ColIndex), 150, 1)

LR分类达到85.22%，MLP分类达到86.33%，SVM分类精度最高，达到86.44%。